In [ ]:
%pylab inline
import os, sys
sys.path.append("../")
import traml
import db
from scipy.optimize import minimize
import copy

In [ ]:
a = traml.Stack()
a.library.search('SiO2')
a.library.search('mgf2')

In [ ]:
PAZO = 500603
PMMA = 500201
sio2 = 111401
mgf2 = 105404
a.library.page(sio2)

In [ ]:
def fun(x, stack, layers, c, delta, TR):
    for i in layers:
        stack.d(i, x[i-1])

    for p in x:
        p = np.sqrt(x[0]**2)
    x[0]=np.sqrt(x[0]**2)
    #min = 1-stack.average(TR, c, delta, o='a')
    min = stack.average(TR, c, delta)
   
    return min**10

In [ ]:
def minTR(stack, film_range, c, delta, TR='T'):
    
    x = []
    layers = []
    bnds = []
    for i in range(film_range[0],film_range[1]+1):
        x.append(stack.config[i][1])
        layers.append(i)
        bnds.append(stack.config[i][3])
        
    result = minimize(fun, x, args=(stack, layers, c,delta, TR), bounds = bnds,tol=1e-30, method='SLSQP')
    
    
    return result

In [ ]:
c = traml.Stack()

peak = 0.65
delta = 0.01

stack_list = []


R=[]
n = [0]

N =40
for i in range(0, N):
    #if i % 2 == 0:
       # print 'even', i
        #c.add(PMMA,100, bnds=(50,250))
    # else:
        #c.add(PAZO,100, bnds=(50,250))
    
    c.add(mgf2, 100, bnds=(0,250))
    c.add(sio2, 100, bnds=(0,250))
    
    
    result = minTR(c, [1,len(c.config)-1], peak, delta, TR='T')
    R.append(c.average('R',peak,delta))
    n.append(i+1)
    
    stack_list.append(copy.deepcopy(c))
  
    print '============================================================================================'
   
    print i+1, ': ', 'R = ', c.average('R',peak,delta)
 
    

In [ ]:
c.plot()

In [ ]:
fig = figure()
ax = fig.add_subplot(111)
R = []

for stack in stack_list:
    R.append(stack.average('R',0.65,0.01))
    
ax.plot(R)
stack_list[-1].average('R', 0.65, 0.01)

In [ ]:
fig = figure()
ax = fig.add_subplot(111)
R = real(c.get_R())
x = c.x

ax.plot(x, R)
ax.set_xlim([0.3,0.9])
    